In [75]:
import numpy as np

%matplotlib ipympl
%matplotlib inline
import matplotlib.pyplot as plt

I don't understand why you need to run this command twice to get an interactive matplotlib display

In [76]:
%matplotlib ipympl

In [77]:
import lsst.daf.persistence        as dafPersist
import lsst.afw.geom               as afwGeom
from matplotlib.colors import LogNorm
import lsst.afw.table              as afwTable
import lsst.meas.extensions.psfex.psfexPsfDeterminer

import lsst.afw.display            as afwDisplay
import lsst.afw.display.utils as afwDisplayUtils

#### Load the high-level "tasks" that process the pixels

In [78]:
from lsst.pipe.tasks.characterizeImage import CharacterizeImageTask

#### Setup the displays (by default an interface to ds9, which won't work in the LSP)

In [79]:
afwDisplay.setDefaultBackend("matplotlib")

## Create the task

In [80]:
config = CharacterizeImageTask.ConfigClass()
config.psfIterations = 1
config.measurePsf.psfDeterminer.name = "psfex"
charImageTask = CharacterizeImageTask(None, config=config)

## Time to process some data

#### Read the input data

In [81]:
butler = dafPersist.Butler("/Users/naka.keigo/Desktop/integ/INTEGRATION/rerun/integration/arc/")

In [82]:
dataId = dict(visit=31)
exposure = butler.get("calexp", visit=31, arm="r",spectrograph=1)
dataRef=butler.dataRef("calexp", visit=31, arm="r", spectrograph=1)


In [83]:
degrees = lsst.geom.degrees
cdMatrix = np.array([[1.0e-4, 0.0], [0.0, 1.0e-4]], dtype=float)
exposure.setWcs(lsst.afw.geom.makeSkyWcs(crval=lsst.geom.SpherePoint(0*degrees, 0*degrees),
                                            crpix=lsst.geom.Point2D(0.0, 0.0),
                                            cdMatrix=cdMatrix))

In [84]:
%matplotlib ipympl

#disp = afwDisplay.Display(reopenPlot=True)
disp = afwDisplay.Display(reopenPlot=False, fastMaskDisplay=True)
disp.scale('asinh', -20, 30, Q=8)
disp.mtv(exposure, title=dataId)

disp.zoom(8, 500, 1500);

FigureCanvasNbAgg()

### Process the pixels

##### Characterise the exposure (e.g. estimate the PSF)

In [85]:
# comparison of images
plt.figure(figsize=(10,5))

plt.subplot(161)
plt.imshow(exposure.getImage().array[100:400,445:465],origin='lower',norm=LogNorm())

plt.subplot(163)
plt.imshow(exposure.getImage().array[2068:2168,1987:1987+9],origin='lower',norm=LogNorm())

plt.subplot(162)
plt.imshow(exposure.getImage().array[68+4000-100:68+4000,1987:1987+9],origin='lower',norm=LogNorm())


FigureCanvasNbAgg()

In [86]:
from lsst.obs.base import ExposureIdInfo
result = charImageTask.detectMeasureAndEstimatePsf(exposure,ExposureIdInfo(), None)
#psf = exposure.getPsf()

In [87]:
#result = charImageTask.run(dataRef, exposure, doUnpersist=False)
#result = charImageTask.run(exposure)

In [88]:
maskedImage=exposure.getMaskedImage()
psf = exposure.getPsf()
w, h = exposure.getDimensions()

In [89]:
box=exposure.getImage().array[1086:1127,441:482]
normbox = (box-box.min())/(box.max()-box.min())
print(box)

[[ -4.91534561e-01  -4.59154272e+00  -3.79155064e+00 ...,   4.20815516e+00
    4.08146679e-01   3.00813818e+00]
 [  8.02903250e-03   2.78830081e-01  -1.09198689e+00 ...,  -2.49228048e+00
   -9.49228859e+00   3.07703078e-01]
 [  3.10766578e+00  -2.33001685e+00   7.65002891e-03 ...,   3.30735755e+00
   -3.09265089e+00  -2.19265914e+00]
 ..., 
 [  9.92149830e-01   1.59214306e+00   5.19213629e+00 ...,   5.19188213e+00
    4.09187460e+00   5.29186773e+00]
 [ -1.36548281e+00  -2.08292931e-01   2.29170012e+00 ...,  -2.08552822e-01
   -1.60855997e+00   1.99143279e+00]
 [ -5.20872211e+00   8.91270936e-01   1.29126406e+00 ...,  -4.80898809e+00
    3.44452083e-01  -3.87869620e+00]]


In [90]:
psfplt= psf.computeImage(afwGeom.PointD(1106, 461)).convertF()
normpsf = (psfplt.array-psfplt.array.min())/(psfplt.array.max()-psfplt.array.min())
print(normpsf)


[[ 0.00136017  0.00136017  0.00136017 ...,  0.00136017  0.00136017
   0.00136017]
 [ 0.00136017  0.00136017  0.00136017 ...,  0.00136017  0.00136017
   0.00136017]
 [ 0.00136017  0.00136017  0.00136017 ...,  0.00136017  0.00136017
   0.00136017]
 ..., 
 [ 0.00136017  0.00136017  0.00136017 ...,  0.00136017  0.00136017
   0.00136017]
 [ 0.00136017  0.00136017  0.00136017 ...,  0.00136017  0.00136017
   0.00136017]
 [ 0.00136017  0.00136017  0.00136017 ...,  0.00136017  0.00136017
   0.00136017]]


In [91]:
%matplotlib ipympl

import matplotlib.gridspec as gridspec

t = np.linspace(-20, 20, 41)
f = np.linspace(-20, 20, 41)

flim = (f.min(), f.max())
tlim = (t.min(), t.max())

plt.figure(figsize=(6,5))

gs = gridspec.GridSpec(2, 2, width_ratios=[1,3], height_ratios=[3,1])
ax = plt.subplot(gs[0,1])
axl = plt.subplot(gs[0,0], sharey=ax)
axb = plt.subplot(gs[1,1], sharex=ax)

ax.imshow(normpsf, origin='lower', extent=tlim+flim, aspect='auto')
plt.xlim(tlim)

axl.plot(normpsf.mean(1),f )
axb.plot(t, normpsf.mean(0))

plt.figure(figsize=(6,5))

gs2 = gridspec.GridSpec(2, 2, width_ratios=[1,3], height_ratios=[3,1])
ax2 = plt.subplot(gs[0,1])
axl2 = plt.subplot(gs[0,0], sharey=ax)
axb2 = plt.subplot(gs[1,1], sharex=ax)

ax2.imshow(normbox, origin='lower', extent=tlim+flim, aspect='auto')
plt.xlim(tlim)

axl2.plot(normbox.mean(1),f )
axb2.plot(t, normbox.mean(0))

plt.figure(figsize=(6,5))

plt.plot(normbox.mean(0))
plt.plot(normpsf.mean(0))


plt.figure(figsize=(6,5))
plt.imshow(normpsf-normbox, origin='lower', extent=tlim+flim, aspect='auto')

FigureCanvasNbAgg()

FigureCanvasNbAgg()

FigureCanvasNbAgg()

FigureCanvasNbAgg()

## Look at the results

In [87]:
exposure.getDimensions()

Extent2I(4096, 4176)

In [43]:
%matplotlib ipympl

mos = afwDisplayUtils.Mosaic()

for y in np.linspace(0, h, 10):
    for x in np.linspace(0, w, 10):
        im = psf.computeImage(afwGeom.PointD(x, y)).convertF()
        mos.append(im)

disp2 = afwDisplay.Display(2, reopenPlot=True)
disp2.scale('asinh', 'zscale')
mos.makeMosaic(display=disp2);

FigureCanvasNbAgg()

In [18]:
plt.imshow(psf.computeImage().array[100:400,445:465],origin='lower',norm=LogNorm())

ValueError: zero-size array to reduction operation minimum which has no identity

In [ ]:
disp2 = afwDisplay.Display(2, reopenPlot=True)

disp2.scale('asinh', 'zscale', Q=4)
disp2.mtv(exposure, title='post calib')

# Overlay something about the measurements
disp2.erase()
with disp2.Buffering():
    for s in sources[good]:
        disp2.dot('+', *s.getCentroid(),
                  ctype=afwDisplay.GREEN if s.get("parent") == 0 else afwDisplay.CYAN)
        
# and some apertures used for photometry
radii = algMetadata.getArray("base_CircularApertureFlux_radii")

with disp2.Buffering():
    for s in sources[good]:
        if exposure.getPhotoCalib().instFluxToMagnitude(s, 'slot_ApFlux').value < 20.0:
            for radius in radii:
                disp2.dot('o', *s.getCentroid(), size=radius, ctype=afwDisplay.YELLOW)